In [1]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
# ___library_import_statements___
import pandas as pd
# make pandas to print dataframes nicely
pd.set_option('expand_frame_repr', False)  
#import pandas_datareader.data as web
import matplotlib.pyplot as plt
#import datetime
import numpy as np
import time

In [2]:
#newest yahoo API 
#!pip uninstall --yes yfinance
!pip install yfinance 
import yfinance as yf
%matplotlib inline

In [3]:
#start_time = datetime.datetime(2017, 10, 1)
#end_time = datetime.datetime(2019, 1, 20)
#end_time = datetime.datetime.now().date().isoformat()         
#today = time.time()

In [4]:
# yahoo gives only daily historical data
def get_data(ticker):
    connected = False
    while not connected:
        try:
            df = yf.download(ticker, end=time.time(), period="5y", interval="1d", group_by = 'ticker', auto_adjust = False, timeout=5.0)
            connected = True
            print('connected to yahoo')
        except Exception as e:
            print("type error: " + str(e))
            time.sleep( 5 )
            pass   
    return df    

In [5]:
def computeRSI(data, time_window):
    df = pd.DataFrame(data)
    df.columns = ['Close']
    df['change'] = df['Close'].diff()

    df['gain'] = df['change'].where(df['change'] > 0, 0.0)
    df['loss'] = -df['change'].where(df['change'] < 0, 0.0)

    # Initialize avg_gain and avg_loss
    df['avg_gain'] = np.nan
    df['avg_loss'] = np.nan

    n = time_window

    # Set the first average gain/loss
    df.at[df.index[n], 'avg_gain'] = df['gain'].iloc[1:n+1].mean()
    df.at[df.index[n], 'avg_loss'] = df['loss'].iloc[1:n+1].mean()

    # Calculate the rest with smoothing
    for i in range(n+1, len(df)):
        df.at[df.index[i], 'avg_gain'] = (df.at[df.index[i-1], 'avg_gain'] * (n - 1) + df.at[df.index[i], 'gain']) / n
        df.at[df.index[i], 'avg_loss'] = (df.at[df.index[i-1], 'avg_loss'] * (n - 1) + df.at[df.index[i], 'loss']) / n

    df['rs'] = df['avg_gain'] / df['avg_loss']
    df['rsi'] = 100 - (100 / (1 + df['rs']))
    return df['rsi']


In [6]:
def computeRSI2(data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [7]:
result = pd.DataFrame(index=np.arange(0, 50), columns=['Date', 'Ticker', 'Close', 'RSI_w', 'RSI_m'])

# ___variables___
ticker = ['BABA',
          '9988.HK',    
          '2269.HK',
          '3690.HK',
          'OKTA',
          'TWLO',
          'TXG',
          'SE',
          'BZUN',
          '9991.HK',
          '2158.HK',
          '0909.HK',
          '9698.HK',
          '600276.SS',        
          '0066.HK',
          '0003.HK',
          '0270.HK',
          '0823.HK',
          '1044.HK',
          '0001.HK',
          '2318.HK',
          '0914.HK',
          '0124.HK',
          '3141.HK' 
         ]

In [8]:
for i in range(len(ticker)):

    # run the function, start="2020-03-01", end="2025-03-01"
    df = yf.download(ticker[i], period="5y",auto_adjust=False) 
        
    # Flatten MultiIndex columns by joining levels with a space
    df.columns = [''.join(col).strip() for col in df.columns]
    
    # Remove the ticker symbol from column names
    df.columns = df.columns.str.replace(f'{ticker[i]}', '', regex=False)
    
    # resample with dictionary
    # each column has to be handled differently
    agg_dict = {'Open': 'first',
              'High': 'max',
              'Low': 'min',
              'Close': 'last',
             # 'Adj Close': 'last',
              'Volume': 'mean'}

    rw_df = df.resample('W').agg(agg_dict)
    rw_df['RSI_w'] = computeRSI(rw_df['Close'], 14)

    rm_df = df.resample('ME').agg(agg_dict)
    rm_df['RSI_m'] = computeRSI(rm_df['Close'], 14)
        
    result.loc[i] = [df.index[-1].date(), 
                 ticker[i], 
                 "%.2f" % df['Close'].iloc[-1], 
                 round(rw_df['RSI_w'].iloc[-1], 0),
                 round(rm_df['RSI_m'].iloc[-1], 0)]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
result

,Date,Ticker,Close,RSI_w,RSI_m
0,2025-05-30,BABA,113.84,50.0,55.0
1,2025-05-30,9988.HK,113.90,52.0,56.0
2,2025-05-30,2269.HK,24.95,59.0,47.0
3,2025-05-30,3690.HK,138.00,45.0,50.0
4,2025-05-30,OKTA,103.17,51.0,53.0
5,2025-05-30,TWLO,117.70,59.0,57.0
6,2025-05-30,TXG,9.53,40.0,34.0
7,2025-05-30,SE,160.37,69.0,73.0
8,2025-05-30,BZUN,2.85,49.0,38.0
9,2025-05-30,9991.HK,7.54,50.0,40.0


In [17]:
result = pd.DataFrame(index=np.arange(0, 50), columns=['Date', 'Ticker', 'Close', 'RSI_w', 'RSI_m'])

# ___variables___
ticker = ['515800.SS',
          '159633.SZ',
          'BND',
          'BNDX',
          'GLDM', 
          'BCI',
          'SGOV',
          '3053.HK',
          # '511360.SS',
          '3110.HK',
          # '563020.SS',
          '515180.SS',
          'IJR',
          'VNQ',
          'VNQI',
          '512200.SS',
          'VT',
          'BNDW',
          'REET',
          'USFR',
          # '511880.SS'
         ]

In [18]:
for i in range(len(ticker)):

    # run the function, start="2020-03-01", end="2025-03-01"
    df = yf.download(ticker[i], period="5y",auto_adjust=False) 
        
    # Flatten MultiIndex columns by joining levels with a space
    df.columns = [''.join(col).strip() for col in df.columns]
    
    # Remove the ticker symbol from column names
    df.columns = df.columns.str.replace(f'{ticker[i]}', '', regex=False)
    
    # resample with dictionary
    # each column has to be handled differently
    agg_dict = {'Open': 'first',
              'High': 'max',
              'Low': 'min',
              'Close': 'last',
             # 'Adj Close': 'last',
              'Volume': 'mean'}

    rw_df = df.resample('W').agg(agg_dict)
    rw_df['RSI_w'] = computeRSI(rw_df['Close'], 14)

    rm_df = df.resample('ME').agg(agg_dict)
    rm_df['RSI_m'] = computeRSI(rm_df['Close'], 14)
        
    result.loc[i] = [df.index[-1].date(), 
                 ticker[i], 
                 "%.2f" % df['Close'].iloc[-1], 
                 round(rw_df['RSI_w'].iloc[-1], 0),
                 round(rm_df['RSI_m'].iloc[-1], 0)]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [19]:
result

,Date,Ticker,Close,RSI_w,RSI_m
0,2025-05-30,515800.SS,1.00,51.0,51.0
1,2025-05-30,159633.SZ,2.48,52.0,62.0
2,2025-05-30,BND,72.77,50.0,46.0
3,2025-05-30,BNDX,49.45,54.0,49.0
4,2025-05-30,GLDM,65.24,65.0,85.0
5,2025-05-30,BCI,20.30,47.0,47.0
6,2025-05-30,SGOV,100.71,56.0,56.0
7,2025-05-30,3053.HK,1158.60,80.0,100.0
8,2025-05-30,3110.HK,26.24,65.0,65.0
9,2025-05-30,515180.SS,1.36,55.0,54.0
